# Data science with IBM Planning Analytics
# Cubike example - Part 1


Cubike is a fictional Bike Sharing company that we use in the series of articles about Data Science with TM1 and Planning Analytics:

* [Part 1: Upload weather data from web services](https://code.cubewise.com/tm1py-help-content/upload-weather-data-from-web-services-into-planning-analytics)
* [Part 2: Explore your TM1 and Planning Analytics data with Pandas and Ploty](https://code.cubewise.com/tm1py-help-content/explore-you-tm1-planning-analytics-with-pandas-and-ploty)
* [Part 3: Timeseries Forecasting with Facebook Prophet](https://code.cubewise.com/tm1py-help-content/timeseries-forecasting-with-facebook-prophet-and-tm1-planning-analytics)
If you are new to TM1py, this article will guide you to setup TM1py and all the different modules required in the Cubike example.

**Note**: All the prerequisites to run this sample into your environement are defined in this article:
* [Setup cubike example](https://code.cubewise.com/tm1py-help-content/setup-cubike-example)


# Part 1: Upload weather data from web services into TM1/Planning Analytics
The objective in this first part is to upload weather data from [National Oceanic and Atmospheric Administation (NOAA) Web service API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

## Step 1: Imports librairies

The first step in the Jupyter notebook is to import the packages and define the constants we need:

* **requests**: library for HTTP / REST Request against Webservices

* **json**: standard library for JSON parsing, manipulation


In [ ]:
import requests
import json
from TM1py import TM1Service

## Constants

We are pulling the weather data from the National Oceanic and Atmospheric Administation (NOAA). NOAA has a rich API which allows us to access all kind of environmental data from the US.

<b>STATION</b>  <a href="https://www.ncdc.noaa.gov/cdo-web/datasets/NORMAL_DLY/stations/GHCND:USW00014732/detail">GHCND:USW00014732</a> (40.7792°, -73.88°) 

<b>FROM, TO</b> Timeframe

<b>HEADERS</b> Token for Authentication with the API

In [ ]:
STATION = 'GHCND:USW00014732'
FROM, TO = '2017-01-01', '2017-01-04'
HEADERS = {"token": 'yyqEBOAbHVbtXkfAmZuPNfnSXvdfyhgn'}

## Step 2: Build URL for the Query

Build the parametrized URL and print it


In [ ]:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?' \
      'datasetid=GHCND&' \
      'startdate=' + FROM + '&' \
      'enddate=' + TO + '&' \
      'limit=1000&' \
      'datatypeid=TMIN&' \
      'datatypeid=TAVG&' \
      'datatypeid=TMAX&' \
      'stationid=' + STATION

print(url)

This is the URL we will get the data from.

## Step 3: Query Weather Data

Now that our URL is ready, we need to send the request to the API:

* Execute the URL against the NOAA API to get the results
* Prettyprint first three items from result-set

In [ ]:
response = requests.get(url, headers=HEADERS).json()
results = response["results"]   

print(json.dumps(results[0:3], indent=2))

## Step 4: Rearrange Data 

Before sending the data into TM1, we now need to rearrange the data so it matches our TM1 cube structure:

* Version = Actual
* Date = record['date'][0:10]
* City = NYC
* DataType = record['datatype']

In [ ]:
cells = dict()

for record in results:
    value = record['value'] / 10
    coordinates = ("Actual", record['date'][0:10], "NYC", record['datatype'])
    cells[coordinates] = value

In [ ]:
for coordinate, value in cells.items():
    print(coordinate, value)

## Step 5: Send Data to TM1
Now that the data is ready, we just need to connect to our TM1 instance and finally write the values into the TM1 cube "Weather Data".

First we need to get the authentication parameters of our TM1 instance which are stored in a config.ini file:


In [ ]:
import configparser
config = configparser.ConfigParser()
config.read(r'..\..\config.ini')

With TM1py we can send data to a cube with two lines of code as long as our cell set match dimensions in our cube:

In [ ]:
with TM1Service(**config['tm1srv01']) as tm1:
    tm1.cubes.cells.write_values("Weather Data", cells)

# Next
To open next Jupyter Notebook:
1. Go to File at the top left
1. Click Open
1. A new tab will open and then click on cubike_data_science.ipynb